## Module 4 Final Project Submission - Computer Vision

* Student name: Emily J. Cain
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: Rafael Carrasco
* Blog post URL: https://medium.com/@magicmilly


## I. Load & Organize Data
## II. Inspection & Exploratory Data Analysis
## III. Preprocessing
## IV. Modeling 
## V. Model Evaluation
## VI. Conclusions & Recommendations

In [4]:
import datetime
import numpy as np
import pandas as pd
import time
import warnings
# import zipfile

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings(action='ignore', category=FutureWarning)